In [1]:
import torch
import pandas as pd

/opt/homebrew/Caskroom/miniconda/base/envs/snu-datascience/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 클래스 정의
class LoanAnalysis:
    """
    1. 라벨링 수행 TN, FN, TP, FP
    2. 각 라벨별 annualized return(ar) 계산 수행: annualized return = return^(1/term)
    3. 포트폴리오 전체 ar 계산 수행
    """
    def __init__(self, dataframe):
        self.df = dataframe.copy()
        self.label_results()

    def label_results(self):
        self.df['result'] = self.df.apply(lambda row: 
            'True Negative' if row['predicted'] == 0 and row['actual'] == 0 else 
            'False Negative' if row['predicted'] == 0 and row['actual'] == 1 else 
            'True Positive' if row['predicted'] == 1 and row['actual'] == 1 else 
            'False Positive' if row['predicted'] == 1 and row['actual'] == 0 else 
            'Other', axis=1)

    def calculate_true_negative_ar(self):
        """
        True Negative 레코드를 필터링하고 return 열을 계산합니다.
        """
        true_negative_df = self.df[self.df['result'] == 'True Negative'].copy()
        true_negative_df['revenue'] = true_negative_df['total_pymnt'] # 대출이자수익

        return true_negative_df

    def calculate_false_negative_ar(self):
        """
        False Negative 레코드를 필터링하고 return 열을 계산합니다.
        """
        false_negative_df = self.df[self.df['result'] == 'False Negative'].copy()
        false_negative_df['revenue'] = false_negative_df['total_pymnt'] - false_negative_df['loan_amnt'] + false_negative_df['recoveries'] - false_negative_df['collection_recovery_fee'] # default로 인한 손실
        return false_negative_df

    def calculate_positives_ar(self):
        """
        True Positive와 False Positive 레코드를 필터링하고 return 열을 계산합니다.
        """
        positive_df = self.df[self.df['result'].isin(['True Positive', 'False Positive'])].copy()
        positive_df['revenue'] = positive_df['tbond_int'] * positive_df['loan_amnt'] + positive_df['loan_amnt'] # 국채수익
        return positive_df

    def process_all(self):
        """
        모든 레코드에 대한 return 열을 계산합니다.
        """
        tn_df = self.calculate_true_negative_ar()
        fn_df = self.calculate_false_negative_ar()
        pos_df = self.calculate_positives_ar()

        # 각 데이터프레임을 결합하여 반환
        final_df = pd.concat([tn_df, fn_df, pos_df])
        return final_df
    
    def calculate_portfolio_ar(self):
        final_df = self.process_all()
        portfolio_annualized_return = ((final_df['revenue'].sum()/ final_df['loan_amnt'].sum())**(1/5)-1)*100
        return portfolio_annualized_return


In [7]:
# 예시 데이터프레임 생성
data = {
    'predicted': [0, 1, 0, 1],
    'actual': [0, 1, 1, 0],
    'term': [36, 60, 36, 60],
    'total_pymnt': [1000, 0, 300, 0],
    'loan_amnt': [1000, 1000, 1000, 1000],
    'recoveries': [0, 0, 100, 0],
    'collection_recovery_fee': [0, 0, 10, 0],
    'tbond_int': [0, 0.01, 0, 0.02],
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# 클래스 인스턴스 생성
analysis = LoanAnalysis(df)

# 모든 데이터를 처리하고 결과를 얻음
final_result_df = analysis.process_all()
final_result_df

,predicted,actual,term,total_pymnt,loan_amnt,recoveries,collection_recovery_fee,tbond_int,result,revenue
0,0,0,36,1000,1000,0,0,0.00,True Negative,1000.0
2,0,1,36,300,1000,100,10,0.00,False Negative,-610.0
1,1,1,60,0,1000,0,0,0.01,True Positive,1010.0
3,1,0,60,0,1000,0,0,0.02,False Positive,1020.0


In [10]:
portfolio_annualized_return = analysis.calculate_portfolio_ar()
print(f"portfolio annualized return : {portfolio_annualized_return:.2f}%")

portfolio annualized return : -9.56%
